In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import random
import math
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import re


/home/venkat/anaconda3/envs/multihead/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
xls = pd.ExcelFile('Master Data Sheet Organization.xlsx')
data_df = pd.read_excel(xls, 'Master Sheet - M1')
new_header = data_df.iloc[0] #grab the first row for the header
data_df = data_df[1:] #take the data less the header row
data_df.columns = new_header #set the header row as the df header
data_df = data_df.loc[(data_df['Picture?'] == 'Y') & (data_df['Morph ID Genus'] != 'Culex')].reset_index(drop=True)
data_df.reset_index(drop=True, inplace=True)
data_df

,NaN,Country,District,Sentinel Site,Date,MOSQUITO ID (Team),Morph ID Genus,Morph ID Species,PCR Number (Country Specific),PCR ID Genus,PCR ID Species,Sibling Species,Picture?,Prototype,Abdomen Status,Sex,Parity,Eliza Results
0,6.0,Uganda,Soroti,Awoja,2021-08-13 00:00:00,U-SR-01-0006,Anopheles,funestus,498,NaN,NaN,NaN,Y,1409,SG,F,NaN,NaN
1,8.0,Uganda,Soroti,Awoja,2021-08-13 00:00:00,U-SR-01-0008,Anopheles,funestus,NaN,Anopheles,funestus,ss,Y,NaN,UF,NaN,NaN,NEG
2,9.0,Uganda,Soroti,Awoja,2021-08-13 00:00:00,U-SR-01-0009,Anopheles,funestus,NaN,NaN,NaN,NaN,Y,NaN,G,F,NaN,NEG
3,10.0,Uganda,Soroti,Awoja,2021-08-13 00:00:00,U-SR-01-0010,Anopheles,funestus,NaN,Anopheles,funestus,ss,Y,NaN,G,F,NaN,NEG
4,11.0,Uganda,Soroti,Awoja,2021-08-13 00:00:00,U-SR-01-0011,Anopheles,funestus,NaN,Anopheles,funestus,ss,Y,NaN,NaN,NaN,NaN,NEG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1180,1646.0,Zambia,Matcha,B2,2021-05-10 00:00:00,Z-MA-01-0295,Anopheles,gambiae,MI2100295,NaN,NaN,NaN,Y,NaN,UF,F,NaN,NaN
1181,1647.0,Zambia,Matcha,B2,2021-05-10 00:00:00,Z-MA-01-0296,Anopheles,rufipes,MI2100296,Anopheles,gambiae,arabiensis,Y,NaN,UF,F,NaN,NaN
1182,1648.0,Zambia,Matcha,Chidakwa,2021-05-10 00:00:00,Z-MA-01-0297,Anopheles,rufipes,MI2100297,NaN,NaN,NaN,Y,NaN,UF,F,NaN,NaN
1183,1649.0,Zambia,Matcha,Chidakwa,2021-05-09 00:00:00,Z-MA-01-0298,Anopheles,longipalps,MI2100298,NaN,NaN,NaN,Y,NaN,UF,F,NaN,NaN


In [3]:
# selecting rows based on condition
df1 = data_df.loc[(data_df["Abdomen Status"] == "UF")]
df2 = data_df.loc[(data_df["Abdomen Status"] == "FF")]
df3 = data_df.loc[(data_df["Abdomen Status"] == "SG")]
df4 = data_df.loc[(data_df["Abdomen Status"] == "G")]

print(df1.shape)
print(df2.shape)
print(df3.shape)
print(df4.shape)

(478, 18)
(162, 18)
(61, 18)
(76, 18)


In [4]:

df1

,NaN,Country,District,Sentinel Site,Date,MOSQUITO ID (Team),Morph ID Genus,Morph ID Species,PCR Number (Country Specific),PCR ID Genus,PCR ID Species,Sibling Species,Picture?,Prototype,Abdomen Status,Sex,Parity,Eliza Results
1,8.0,Uganda,Soroti,Awoja,2021-08-13 00:00:00,U-SR-01-0008,Anopheles,funestus,NaN,Anopheles,funestus,ss,Y,NaN,UF,NaN,NaN,NEG
14,21.0,Uganda,Soroti,Awoja,2021-08-13 00:00:00,U-SR-01-0021,Anopheles,funestus,NaN,Anopheles,funestus,ss,Y,NaN,UF,NaN,NaN,NEG
15,22.0,Uganda,Soroti,Awoja,2021-08-13 00:00:00,U-SR-01-0022,Anopheles,funestus,NaN,Anopheles,funestus,ss,Y,NaN,UF,NaN,NaN,NEG
16,24.0,Uganda,Soroti,Awoja,2021-08-13 00:00:00,U-SR-01-0024,Anopheles,funestus,NaN,Anopheles,funestus,ss,Y,NaN,UF,NaN,NaN,NEG
17,25.0,Uganda,Soroti,Awoja,2021-08-13 00:00:00,U-SR-01-0025,Anopheles,funestus,NaN,Anopheles,funestus,ss,Y,NaN,UF,NaN,NaN,NEG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1180,1646.0,Zambia,Matcha,B2,2021-05-10 00:00:00,Z-MA-01-0295,Anopheles,gambiae,MI2100295,NaN,NaN,NaN,Y,NaN,UF,F,NaN,NaN
1181,1647.0,Zambia,Matcha,B2,2021-05-10 00:00:00,Z-MA-01-0296,Anopheles,rufipes,MI2100296,Anopheles,gambiae,arabiensis,Y,NaN,UF,F,NaN,NaN
1182,1648.0,Zambia,Matcha,Chidakwa,2021-05-10 00:00:00,Z-MA-01-0297,Anopheles,rufipes,MI2100297,NaN,NaN,NaN,Y,NaN,UF,F,NaN,NaN
1183,1649.0,Zambia,Matcha,Chidakwa,2021-05-09 00:00:00,Z-MA-01-0298,Anopheles,longipalps,MI2100298,NaN,NaN,NaN,Y,NaN,UF,F,NaN,NaN


In [5]:
# one train and test split
# from sklearn.model_selection import train_test_split

# train_data1, test_data1 = train_test_split(df1, test_size=0.15)
# train_data2, test_data2 = train_test_split(df2, test_size=0.15)
# train_data3, test_data3 = train_test_split(df3, test_size=0.15)
# train_data4, test_data4 = train_test_split(df4, test_size=0.15)

In [6]:
train_data = pd.concat(
    [df1,df2,df3,df4], ignore_index=True
)
# test_data = pd.concat(
#     [test_data1, test_data2, train_data3, train_data4], ignore_index=True
# )
print(train_data.shape)
# print(test_data.shape)


(777, 18)


In [7]:
train_data

,NaN,Country,District,Sentinel Site,Date,MOSQUITO ID (Team),Morph ID Genus,Morph ID Species,PCR Number (Country Specific),PCR ID Genus,PCR ID Species,Sibling Species,Picture?,Prototype,Abdomen Status,Sex,Parity,Eliza Results
0,8.0,Uganda,Soroti,Awoja,2021-08-13 00:00:00,U-SR-01-0008,Anopheles,funestus,NaN,Anopheles,funestus,ss,Y,NaN,UF,NaN,NaN,NEG
1,21.0,Uganda,Soroti,Awoja,2021-08-13 00:00:00,U-SR-01-0021,Anopheles,funestus,NaN,Anopheles,funestus,ss,Y,NaN,UF,NaN,NaN,NEG
2,22.0,Uganda,Soroti,Awoja,2021-08-13 00:00:00,U-SR-01-0022,Anopheles,funestus,NaN,Anopheles,funestus,ss,Y,NaN,UF,NaN,NaN,NEG
3,24.0,Uganda,Soroti,Awoja,2021-08-13 00:00:00,U-SR-01-0024,Anopheles,funestus,NaN,Anopheles,funestus,ss,Y,NaN,UF,NaN,NaN,NEG
4,25.0,Uganda,Soroti,Awoja,2021-08-13 00:00:00,U-SR-01-0025,Anopheles,funestus,NaN,Anopheles,funestus,ss,Y,NaN,UF,NaN,NaN,NEG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
772,1322.0,NaN,Tamale,Christina Phone,NaN,G-TM-05-0039,Anopheles,gambiae,NaN,NaN,NaN,NaN,Y,NaN,G,F,NaN,NaN
773,1323.0,NaN,Tamale,Christina Phone,NaN,G-TM-05-0040,Anopheles,gambiae,NaN,NaN,NaN,NaN,Y,NaN,G,F,NaN,NaN
774,1326.0,NaN,Tamale,Christina Phone,NaN,G-TM-05-0043,Anopheles,gambiae,NaN,NaN,NaN,NaN,Y,NaN,G,F,NaN,NaN
775,1347.0,NaN,NaN,NaN,NaN,G-TM-05-0064,Anopheles,gambiae,NaN,NaN,NaN,NaN,Y,NaN,G,F,NaN,NaN


In [8]:
from torchvision import transforms
mosquito_transforms = transforms.Compose([
    #transforms.ToPILImage(),
    transforms.Resize([299,299]),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

In [9]:
from PIL import Image

# read data function
def make_M1_data(d):
    data_path = "M1_Database_Cropped"
    data = []
    label = []
    paths = []
    df = pd.DataFrame()
    UF_count = 0
    FF_count = 0
    SG_count = 0
    G_count = 0
    F_count = 0
    M_count = 0
    for i in range(d.shape[0]):
        for f in os.listdir(data_path):
            if re.match(d.loc[i, "MOSQUITO ID (Team)"], f) and f.endswith("a.jpg"):
                path = os.path.join(data_path, f)
                paths.append(path)
                image = Image.open(path)
                image = mosquito_transforms(image)
                data.append(image)
    
                if d.loc[i, "Abdomen Status"] == "SG":
                    label.append(1)
                    SG_count += 1
                if d.loc[i, "Abdomen Status"] == "G":
                    label.append(1)
                    G_count += 1
                if d.loc[i, "Abdomen Status"] == "UF":
                    label.append(0)
                    UF_count += 1
                if d.loc[i, "Abdomen Status"] == "FF":
                    label.append(1)
                    FF_count += 1
    stacked_data = torch.stack(data)
    df["path"] = paths
    df["label"] = label
    print("Counts:", "UF_count:", UF_count, "FF_count:", FF_count, "SG_count:", SG_count, "G_count:", G_count) 


    return stacked_data, label, df

In [11]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
skf = StratifiedKFold(n_splits=5, shuffle = True)
target = train_data.loc[:,'Abdomen Status']

target_list=target.values.tolist()

count = 1
for train_index, test_index in skf.split(train_data, target):
    train_val_target_list=[]
    for i in train_index:
        train_val_target_list.append(target[i])
 
    train = train_data.loc[train_index,:]
    test = train_data.loc[test_index,:]
    
    train = train.reset_index(drop=True)
    test = test.reset_index(drop=True)
    
    # one train and test split

    train_data_split, val_data_split = train_test_split(train, test_size=0.20,stratify=train_val_target_list)
    print(train_data_split["Abdomen Status"])
    
    train_data_split = train_data_split.reset_index(drop=True)
    val_data_split = val_data_split.reset_index(drop=True)
    
    print("Validation: ")
    d, label, path_df = make_M1_data(val_data_split)
    torch.save(d,f"data/abdomen/task1_woculex_cropped_kfold/CV_1_M1/val_data_fold{count}.pt")
    torch.save(torch.tensor(label),f"data/abdomen/task1_woculex_cropped_kfold/CV_1_M1/val_label_fold{count}.pt")
    path_df.to_csv(f"data/abdomen/task1_woculex_cropped_kfold/CV_1_M1/val_datapath_fold{count}.csv", index = False)
    print("Train: ")
    d2, label2, path_df2 = make_M1_data(train_data_split)
    torch.save(d2,f"data/abdomen/task1_woculex_cropped_kfold/CV_1_M1/train_data_fold{count}.pt")
    torch.save(torch.tensor(label2),f"data/abdomen/task1_woculex_cropped_kfold/CV_1_M1/train_label_fold{count}.pt")
    path_df2.to_csv(f"data/abdomen/task1_woculex_cropped_kfold/CV_1_M1/train_datapath_fold{count}.csv", index = False)
    
    print("Test: ")
    d, label, path_df = make_M1_data(test)
    torch.save(d,f"data/abdomen/task1_woculex_cropped_kfold/CV_1_M1/test_data_fold{count}.pt")
    torch.save(torch.tensor(label),f"data/abdomen/task1_woculex_cropped_kfold/CV_1_M1/test_label_fold{count}.pt")
    path_df.to_csv(f"data/abdomen/task1_woculex_cropped_kfold/CV_1_M1/test_datapath_fold{count}.csv", index = False)
    
    count += 1

213    UF
451    FF
514    SG
5      UF
346    UF
       ..
467    FF
70     UF
118    UF
90     UF
48     UF
Name: Abdomen Status, Length: 496, dtype: object
Validation: 
Counts: UF_count: 146 FF_count: 105 SG_count: 37 G_count: 41
Train: 
Counts: UF_count: 480 FF_count: 306 SG_count: 123 G_count: 155
Test: 
Counts: UF_count: 177 FF_count: 103 SG_count: 42 G_count: 50
136    UF
605     G
568     G
77     UF
277    UF
       ..
3      UF
140    UF
247    UF
176    UF
58     UF
Name: Abdomen Status, Length: 496, dtype: object
Validation: 
Counts: UF_count: 114 FF_count: 71 SG_count: 34 G_count: 31
Train: 
Counts: UF_count: 505 FF_count: 329 SG_count: 122 G_count: 163
Test: 
Counts: UF_count: 184 FF_count: 114 SG_count: 46 G_count: 52
408    FF
516    SG
445    FF
176    UF
510    FF
       ..
425    FF
446    FF
115    UF
256    UF
583     G
Name: Abdomen Status, Length: 497, dtype: object
Validation: 
Counts: UF_count: 145 FF_count: 86 SG_count: 30 G_count: 45
Train: 
Counts: UF_count:

# M2

In [20]:
xls = pd.ExcelFile('Master Data Sheet Organization.xlsx')
data_df = pd.read_excel(xls, 'Master Sheet - M2')
new_header = data_df.iloc[0] #grab the first row for the header
data_df = data_df[1:] #take the data less the header row
data_df.columns = new_header #set the header row as the df header
data_df.reset_index(drop=True, inplace=True)
data_df

,mosquito_id,Country,City,Site,Date,specimen_image_1,specimen_image_2,specimen_image_3,morph_id_genus,morph_id_genus_other,...,pcr_present,pcr_id_genus,pcr_id_genus_other,pcr_id_aedes_species,pcr_id_anopheles_species,pcr_id_species_other,pcr_id_sibling,additional_note,PARENT_KEY,KEY
0,G031B,Uganda,Bugiri,Buboki,2022-04-02 00:00:00,1648900605929.jpg,1648900637908.jpg,1648900654087.jpg,Anopheles,NaN,...,later,NaN,NaN,NaN,NaN,NaN,NaN,NaN,uuid:cd9fe629-f8a5-4dc0-8e7e-2eee17bf50b9,uuid:cd9fe629-f8a5-4dc0-8e7e-2eee17bf50b9/hh_r...
1,G032B,Uganda,Bugiri,Buboki,2022-04-02 00:00:00,1648900707180.jpg,1648900718477.jpg,1648900736152.jpg,Anopheles,NaN,...,later,NaN,NaN,NaN,NaN,NaN,NaN,NaN,uuid:cd9fe629-f8a5-4dc0-8e7e-2eee17bf50b9,uuid:cd9fe629-f8a5-4dc0-8e7e-2eee17bf50b9/hh_r...
2,G033B,Uganda,Bugiri,Buboki,2022-04-02 00:00:00,1648900875092.jpg,1648900888835.jpg,1648900905444.jpg,Anopheles,NaN,...,later,NaN,NaN,NaN,NaN,NaN,NaN,NaN,uuid:cd9fe629-f8a5-4dc0-8e7e-2eee17bf50b9,uuid:cd9fe629-f8a5-4dc0-8e7e-2eee17bf50b9/hh_r...
3,G034B,Uganda,Bugiri,Buboki,2022-04-02 00:00:00,1648900946763.jpg,1648900961718.jpg,1648900978547.jpg,Anopheles,NaN,...,later,NaN,NaN,NaN,NaN,NaN,NaN,NaN,uuid:cd9fe629-f8a5-4dc0-8e7e-2eee17bf50b9,uuid:cd9fe629-f8a5-4dc0-8e7e-2eee17bf50b9/hh_r...
4,B035B,Uganda,Bugiri,Buboki,2022-04-02 00:00:00,1648901102036.jpg,1648901120450.jpg,1648901136984.jpg,Anopheles,NaN,...,later,NaN,NaN,NaN,NaN,NaN,NaN,NaN,uuid:cd9fe629-f8a5-4dc0-8e7e-2eee17bf50b9,uuid:cd9fe629-f8a5-4dc0-8e7e-2eee17bf50b9/hh_r...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
709,NaN,NaN,NaN,NaN,NaN,1643193598508.jpg,1643193634018.jpg,1643193701849.jpg,Anopheles,\n,...,later,\n,\n,\n,\n,\n,\n,Na,uuid:b8df0317-2b65-4d17-afe9-fd7ed0f52609,uuid:b8df0317-2b65-4d17-afe9-fd7ed0f52609/hh_r...
710,NaN,NaN,NaN,NaN,NaN,1643192970690.jpg,1643193019263.jpg,1643193065582.jpg,Anopheles,\n,...,later,\n,\n,\n,\n,\n,\n,NaN,uuid:958e8fc8-cb3e-4214-930d-80227b14b4b3,uuid:958e8fc8-cb3e-4214-930d-80227b14b4b3/hh_r...
711,NaN,NaN,NaN,NaN,NaN,1643193284792.jpg,1643193320535.jpg,1643193372510.jpg,Anopheles,\n,...,later,\n,\n,\n,\n,\n,\n,Na,uuid:958e8fc8-cb3e-4214-930d-80227b14b4b3,uuid:958e8fc8-cb3e-4214-930d-80227b14b4b3/hh_r...
712,NaN,NaN,NaN,NaN,NaN,1642774690032.jpg,1642774707378.jpg,1642774718884.jpg,Anopheles,\n,...,no,\n,\n,\n,\n,\n,\n,NaN,uuid:8b60ca61-4660-4ff6-af4b-7e1cfd79b967,uuid:8b60ca61-4660-4ff6-af4b-7e1cfd79b967/hh_r...


In [21]:
df1 = data_df[(data_df['specimen_characteristics'] == 'UF') & (data_df['morph_id_genus'] != 'Culex')]
df2 = data_df[(data_df['specimen_characteristics'] == 'FF') & (data_df['morph_id_genus'] != 'Culex')]
df3 = data_df[(data_df['specimen_characteristics'] == 'SG') & (data_df['morph_id_genus'] != 'Culex')]
df4 = data_df[(data_df['specimen_characteristics'] == 'G') & (data_df['morph_id_genus'] != 'Culex')]
print(df1.shape)
print(df2.shape)
print(df3.shape)
print(df4.shape)

(437, 28)
(17, 28)
(25, 28)
(21, 28)


In [22]:
df1 = df1.sample(n=75).reset_index(drop=True)

In [23]:
train_data = pd.concat([df1, df2, df3, df4], ignore_index=True)
print(train_data.shape)
train_data = train_data.sample(frac=1).reset_index(drop=True)

(138, 28)


In [24]:
from PIL import Image, ImageOps
def make_M2_data(d):
    data_path = "M2_Database_Cropped"
    data = []
    label = []
    paths = []
    abdomen_status = []
    sex = []
    F_count = 0
    G_count = 0
    Aother_count = 0
    C_count = 0
    Other_count = 0
    SG_count = 0
    GG_count = 0
    UF_count = 0
    FF_count = 0
    male_count = 0 
    female_count = 0
    df = pd.DataFrame()
    for i in range(d.shape[0]):
        list_specimen = []
        list_specimen.append(d.loc[i,'specimen_image_1'])
        list_specimen.append(d.loc[i,'specimen_image_2'])
        list_specimen.append(d.loc[i,'specimen_image_3'])
        for specimen in list_specimen:
            if specimen in os.listdir(data_path):
                path = os.path.join(data_path, specimen)
                paths.append(path)
                image = Image.open(path)
                #image = ImageOps.exif_transpose(image)
                image = mosquito_transforms(image)
                data.append(image)
#                 if d.loc[i,'label'] == 0:
#                     label.append(0)
#                     F_count += 1
#                 elif d.loc[i,'label'] == 1:
#                     label.append(1)
#                     G_count += 1
#                 elif d.loc[i,'label'] == 2:
#                     label.append(2)
#                     Aother_count += 1    
#                 elif d.loc[i,'label'] == 3:
#                     label.append(3)
#                     C_count += 1
#                 elif d.loc[i,'label'] == 4:
#                     label.append(4)
#                     Other_count += 1
                if d.loc[i,'specimen_characteristics'] == "SG":
                    abdomen_status.append(1)
                    SG_count += 1
                if d.loc[i,'specimen_characteristics'] == "G":
                    abdomen_status.append(1)
                    GG_count += 1
                if d.loc[i,'specimen_characteristics'] == "UF":
                    abdomen_status.append(0)
                    UF_count += 1
                if d.loc[i,'specimen_characteristics'] == "FF":
                    abdomen_status.append(1)
                    FF_count += 1
#                 if d.loc[i,'specimen_gender'] == "F":
#                     sex.append(0)
#                     female_count += 1
#                 if d.loc[i,'specimen_gender'] == "M":
#                     sex.append(1)
#                     male_count += 1
                    
    stacked_data = torch.stack(data)
    df['path'] = paths
#     df['label'] = label
    df['abdomen'] = abdomen_status
#     df['gender'] = sex 
#     print("F_count: ", F_count)
#     print("G_count: ", G_count)
#     print("Aother_count: ", Aother_count)
#     print("C_count: ", C_count)
#     print("Other_count: ", Other_count)
    print("SG_count: ", SG_count)
    print("GG_count: ", GG_count)
    print("UF_count: ", UF_count)
    print("FF_count: ", FF_count)
#     print("female_count: ", female_count)
#     print("male_count: ", male_count)
    d1={'label':label, 'abdomen':abdomen_status, 'gender':sex}
                
    return stacked_data, d1, df

In [31]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split


skf = StratifiedKFold(n_splits=5, shuffle = True)
target = train_data.loc[:,'specimen_characteristics']
target_list=target.values.tolist()

count = 1
for train_index, test_index in skf.split(train_data, target):
    print(f"Fold{count}: ")
    
    train_val_target_list=[]
    for i in train_index:
        train_val_target_list.append(target[i])
        
    train = train_data.loc[train_index,:]
    test = train_data.loc[test_index,:]
    train = train.reset_index(drop=True)
    test = test.reset_index(drop=True)
    
    train_data_split, val_data_split = train_test_split(train, test_size=0.20,stratify=train_val_target_list)
    
    train_data_split = train_data_split.reset_index(drop=True)
    val_data_split = val_data_split.reset_index(drop=True)
    
    
    
    d, dic1, path_df = make_M2_data(val_data_split)
    torch.save(d,f"data/abdomen/task1_woculex_cropped_kfold/CV_1_M2/val_data_fold{count}.pt")
    np.save(f"data/abdomen/task1_woculex_cropped_kfold/CV_1_M2/val_label_fold{count}.npy", dic1)
    path_df.to_csv(f"data/abdomen/task1_woculex_cropped_kfold/CV_1_M2/val_datapath_fold{count}.csv", index = False)
    print("Train: ")
    d2, dic2, path_df2 = make_M2_data(train_data_split)
    torch.save(d2,f"data/abdomen/task1_woculex_cropped_kfold/CV_1_M2/train_data_fold{count}.pt")
    np.save(f"data/abdomen/task1_woculex_cropped_kfold/CV_1_M2/train_label_fold{count}.npy", dic2)
    path_df2.to_csv(f"data/abdomen/task1_woculex_cropped_kfold/CV_1_M2/train_datapath_fold{count}.csv", index = False)
    print("Test: ")
    d2, dic2, path_df2 = make_M2_data(test)
    torch.save(d2,f"data/abdomen/task1_woculex_cropped_kfold/CV_1_M2/test_data_fold{count}.pt")
    np.save(f"data/abdomen/task1_woculex_cropped_kfold/CV_1_M2/test_label_fold{count}.npy", dic2)
    path_df2.to_csv(f"data/abdomen/task1_woculex_cropped_kfold/CV_1_M2/test_datapath_fold{count}.csv", index = False)
    count += 1

Fold1: 
SG_count:  9
GG_count:  6
UF_count:  33
FF_count:  9
Train: 
SG_count:  39
GG_count:  36
UF_count:  129
FF_count:  18
Test: 
SG_count:  12
GG_count:  9
UF_count:  42
FF_count:  6
Fold2: 
SG_count:  12
GG_count:  6
UF_count:  33
FF_count:  3
Train: 
SG_count:  39
GG_count:  36
UF_count:  129
FF_count:  18
Test: 
SG_count:  9
GG_count:  9
UF_count:  42
FF_count:  12
Fold3: 
SG_count:  9
GG_count:  6
UF_count:  33
FF_count:  6
Train: 
SG_count:  36
GG_count:  33
UF_count:  132
FF_count:  24
Test: 
SG_count:  15
GG_count:  12
UF_count:  39
FF_count:  3
Fold4: 
SG_count:  12
GG_count:  9
UF_count:  36
FF_count:  6
Train: 
SG_count:  39
GG_count:  30
UF_count:  126
FF_count:  21
Test: 
SG_count:  9
GG_count:  12
UF_count:  42
FF_count:  6
Fold5: 
SG_count:  12
GG_count:  12
UF_count:  36
FF_count:  6
Train: 
SG_count:  33
GG_count:  30
UF_count:  129
FF_count:  21
Test: 
SG_count:  15
GG_count:  9
UF_count:  39
FF_count:  6
